In [2]:
import os
import sqlite3
import pandas as pd

def convert_csv_to_sqlite(csv_path: str, sqlite_path: str):
    # CSV 파일 로드
    df = pd.read_csv(csv_path)

    # 주소 통합 처리 (둘 중 하나만 존재함)
    df['주소'] = df['소재지도로명주소'].combine_first(df['소재지지번주소'])

    # 사용하지 않는 주소 필드 제거
    df.drop(columns=['소재지도로명주소', '소재지지번주소'], inplace=True)

    # 테이블명 생성 (시군구명 기준)
    if '시군구명' not in df.columns:
        raise ValueError("CSV에 '시군구명' 컬럼이 존재하지 않습니다.")
    
    table_name = df['시군구명'].iloc[0].strip().replace(" ", "")
    
    # DB 연결 및 테이블 생성
    conn = sqlite3.connect(sqlite_path)
    cursor = conn.cursor()

    # 테이블 생성 SQL
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS [{table_name}] (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        가맹점명 TEXT,
        사용가능지역화폐 TEXT,
        시도명 TEXT,
        시군구명 TEXT,
        주소 TEXT,
        업종명 TEXT,
        주요상품 TEXT,
        전화번호 TEXT,
        데이터기준일자 TEXT
    );
    """
    cursor.execute(create_table_sql)

    # 데이터 삽입
    df.to_sql(table_name, conn, if_exists='append', index=False)

    # 인덱스 생성
    for column in ['가맹점명', '업종명', '주소']:
        cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_{column} ON [{table_name}] ({column});")

    conn.commit()
    conn.close()

    print(f"✅ 변환 완료: {csv_path} → {sqlite_path} ({table_name} 테이블)")

# 사용 예시
convert_csv_to_sqlite("전북특별자치도_진안군_지역화폐가맹점_20250128.csv", "지역화폐_가맹점_DB.db")


✅ 변환 완료: 전북특별자치도_진안군_지역화폐가맹점_20250128.csv → 지역화폐_가맹점_DB.db (진안군 테이블)
